# DeepLearning Frameworks
<hr>

It's not practical to implement everything from scratch. Deep learning is now in the phase of doing something with the frameworks and not from scratch to keep on going. Here are some of the leading deep learning frameworks:
- Caffe / Caffe2
- CNTK
- DL4j
- Keras
- Lasagne
- mxnet
- PaddlePaddle
- TensorFlow
- Theano
- Torch/Pytorch

How to choose deep learning framework:
- Ease of programming (development and deployment)
- Running speed
- Truly open (open source with good governance)

In this notebook, we'll explore TensorFlow.

***In TensorFlow you implement only the forward propagation and TensorFlow will do the backpropagation by itself.***

In [5]:
import numpy as np
import tensorflow as tf

## Cost Function Minimization in TensorFlow
<hr>

Consider the following cost function:

$$J(w) = w^2 -10w + 25$$

Since $w^2 -10w + 25 = (w-5)^2$, the value that minimizes $J(w)$ is $w=5$.

In [6]:
## MINIMIZATION V.1

w = tf.Variable(0, dtype=tf.float32)    # creating a variable in tensorflow, w = 0.0
optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)

def train_step():
    with tf.GradientTape() as tape:
        cost = w**2 -10*w + 25
        
    trainable_variables = [w]
    grads = tape.gradient(cost, trainable_variables)
    optimizer.apply_gradients(zip(grads, trainable_variables))

print("Initial value of w before training")
print(w)

train_step()
print("\nValue of w after 1 training step")
print(w)

for i in range(1000):
    train_step()
print("\nValue of w after 1000 training steps")
print(w)

Initial value of w before training
<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.0>

Value of w after 1 training step
<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.09999931>

Value of w after 1000 training steps
<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=5.000001>


In [7]:
## MINIMIZATION V.2

w = tf.Variable(0, dtype=tf.float32)
x = np.array([1.0, -10.0, 25.0], dtype=np.float32)    # controlling the cost function using an array
optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)

def training(x, w, optimizer):
    def cost_fn():
        return x[0] * w ** 2 + x[1] * w + x[2]
    
    for i in range(1000):
        optimizer.minimize(cost_fn, [w])    # the minimize function replaces the tape.gradient functionality
        
    return w

w = training(x, w, optimizer)
print(w)

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=5.000001>
